In [1]:
from os.path import dirname, join
import re

import numpy as np
import pandas as pd

from elvis import datasets

from elvis.io.boem_from_file import (boem_lease_by_owner, 
                                     boem_leases, 
                                     boem_lease_owner,
                                     contours_to_geojson,
                                     get_blocks_by_owner,
                                     get_blocks,
                                     get_current_leases,
                                     load_regional_prod,
                                     load_production_data,
                                     load_well_headers,
                                     load_discoveries)

base_directory = dirname(datasets.__file__)

# supplied data
freeze_data = join(base_directory, 'Freeze_Data\ 12_4_2019')

#
base_directory = dirname(datasets.__file__)

# Stuff I need

1. From production really just need monthly production associated with the 
   region. 
2. I want production by lease block.
3. And I want wells.

Bonus round:

Field, Discoveries.

In [15]:
fields = pd.read_csv(join(freeze_data, "Fields.txt"))

In [30]:
for col in fields.columns:
    print (col)

FID
FABBRV
FCLASS
FSTAT
FD_TYPE
FJURIS
FSTRU
FTRAP1
FTRAP2
FDDATE
FDYEAR
FDDATEH
FDYEARH
WELLAPI
EIAID
PLAREA
ASSESSED
P_J
WDEP
OEURATL
GEURATL
GEEURATL
EEURATL
OCUMATL
GCUMATL
GECUMATL
ECUMATL
ORESATL
GRESATL
GERESATL
ERESATL
OJUSATL
GJUSATL
GEJUSATL
EJUSATL
ODSCATL
GDSCATL
GEDSCATL
EDSCATL
OIP
GIP
ORECO
ORECG
GRECO
GRECG
NCNT
UCNT
SCNT
TCNT
FSDCOUNT
ESAAREA
LON
LAT
FRSTPRODDT
LASTPRODDT
FPOOL
FSAND
OEURRHIST
GEURRHIST
GEEURRHIST
EEURRHIST
OCUMRHIST
GCUMRHIST
GECUMRHIST
ECUMRHIST
ORESRHIST
GRESRHIST
GERESRHIST
ERESRHIST
OCUMCURR
GCUMCURR
GECUMCURR
WCUMCURR
ECUMCURR
OPRODLY
GPRODLY
GEPRODLY
WPRODLY
EPRODLY
FLYPRDWELL
FLYPRDCOMP
FPLAT
FWELL
FCOMPINT
FCOMP
FTEST
FLYNEWWELL
FLYNEWCOMP
FLYNEWPLAT
FLYOWCUT
FLYGWCUT
FLYOPRDVTY
FLYGPRDVTY
APPRWELL
GASINJWELL
WTRINJWELL
FCLASSIFY
FNICKNAME
FNAME
BLKSHAPE


In [26]:
fields[fields.duplicated(subset=["FNICKNAME"])]

,FID,FABBRV,FCLASS,FSTAT,FD_TYPE,FJURIS,FSTRU,FTRAP1,FTRAP2,FDDATE,...,FLYGWCUT,FLYOPRDVTY,FLYGPRDVTY,APPRWELL,GASINJWELL,WTRINJWELL,FCLASSIFY,FNICKNAME,FNAME,BLKSHAPE
4,4,EB197,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,20050324.0,...,0.0,0.0,0.0,3,0,0,Awaiting Production,,East Breaks 197,
17,17,ST267,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,19870811.0,...,0.0,0.0,0.0,9,0,0,Awaiting Production,,South Timbalier 267,
18,18,ST293,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,20070822.0,...,0.0,0.0,0.0,5,0,0,Awaiting Production,,South Timbalier 293,
19,19,VK207,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,20060802.0,...,0.0,0.0,0.0,6,0,0,Awaiting Production,,Viosca Knoll 207,
20,20,VK741,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,19960516.0,...,0.0,0.0,0.0,2,0,0,Awaiting Production,,Viosca Knoll 741,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1455,MO1005,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,20011012.0,...,0.0,0.0,0.0,1,0,0,Awaiting Production,,Mobile 1005,
1456,1456,MU021A,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,19800702.0,...,0.0,0.0,0.0,6,0,0,Awaiting Production,,Mustang Island 021A,
1457,1457,SP073,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,19811210.0,...,0.0,0.0,0.0,4,0,0,Awaiting Production,,South Pass 073,
1458,1458,VK256,NONASSESSED,NONASSESSED,Nonassessed,FEDERAL,,,,19900820.0,...,0.0,0.0,0.0,6,0,0,Awaiting Production,,Viosca Knoll 256,


In [19]:
fields[fields["FABBRV"] == "GC726"]

,FID,FABBRV,FCLASS,FSTAT,FD_TYPE,FJURIS,FSTRU,FTRAP1,FTRAP2,FDDATE,...,FLYGWCUT,FLYOPRDVTY,FLYGPRDVTY,APPRWELL,GASINJWELL,WTRINJWELL,FCLASSIFY,FNICKNAME,FNAME,BLKSHAPE


In [2]:
disco, _ = load_discoveries(freeze_data)

In [8]:
well_headers = load_well_headers(freeze_data)

In [14]:
disco = pd.read_csv(join(freeze_data, "Discoveries.txt"))

In [29]:
disco[disco["DISC_NICK"] == "WARRIOR/TIMON"]

,FID,Lease Number,DISC_NAME,DISC_NICK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,MIDEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
AREABLK,,,,,,,,,,,,,,,,,,,,,
GC519,139,G34984,GC519,WARRIOR/TIMON,http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,0,...,,,,31,The Warrior discovery is located approximately...,,20170728,07/28/2017,-90.219304,27.428828
GC563,139,G34984,GC519,WARRIOR/TIMON,http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,0,...,,,,31,The Warrior discovery is located approximately...,,20170728,07/28/2017,-90.219304,27.428828


In [39]:
# unpack data form the AREABLOCK column with multiple AREABLKS per lease
leni = np.array([len(val["AREABLOCK"].split(",")) for key,val in disco.iterrows()])
indx1 = leni < 2
indxx = leni > 1

disco_1 = disco[indx1].copy()
disco_m = disco[indxx].copy()

In [71]:
disco_m

,FID,LEASENUM,DISC_NAME,DISC_NICK,AREABLOCK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
0,0,G16783,GC727,CAESAR TONGA,"GC726, GC727, GC770, GC683",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,GC640,200,...,07/02/2003,10/28/2003,PA,7,,,20050822,08/22/2005,-90.833332,27.243621
33,33,G12166,MC765,PRINCESS,"MC765, MC766",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC807,0,...,07/22/2002,09/04/2002,COM,5,Princess is Shell's first Gulf of Mexico devel...,Princess includes an intervention system locat...,20020213,02/13/2002,-89.093596,28.203328
56,56,G21738,EW871,EW871,"EW871, EW872",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EW914,0,...,02/12/2001,03/10/2001,COM,8,EW914 is classified as depleted,,20020812,08/12/2002,-90.275729,28.105105
57,57,G21645,EI343,EI343,"EI343, EI344",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EI342,0,...,06/08/2003,,DRL,0,Engineering has begun on the design of a tripo...,,20030811,08/11/2003,-91.426559,28.192847
94,94,G27278,MC519,SANTA CRUZ/SANTIAGO,"MC519, MC563",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC607,0,...,,,,13,Open-hole logging indicated >140ft of net gas ...,Noble operates Santa Cruz with 23.25% working ...,20090402,04/02/2009,-88.236020,28.475236
115,115,G31943,WR095,YUCATAN NORTH,"WR095 , WR096, WR139 , WR140",http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,...,07/21/2012,,,21,INPEX holds a 15% participating interest and ...,,20130806,08/06/2013,-91.593640,26.861107
125,125,G27982,EW834,COELACANTH/HUMMINGBIRD,"EW834,EW790,EW835,MC793",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EW878,64,...,03/26/2010,,TA,15,The field will produce from two geo-pressured ...,Another development well will capture a proven...,20151202,12/02/2015,-89.937492,28.156996
126,126,G24060,MC214,ODD JOB,"MC214,MC215",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC214,0,...,11/08/2014,12/16/2014,ST,8,Technip will use its Deep Blue ultra-deepwater...,,20151218,12/18/2015,-88.066508,28.781713
128,128,G35491,MC208,NEIDERMEYER,"MC208,MC209,MC252,MC253",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477
139,139,G34984,GC519,WARRIOR/TIMON,"GC519, GC563",http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,...,,,,31,The Warrior discovery is located approximately...,,20170728,07/28/2017,-90.219304,27.428828


In [131]:
disco = pd.read_csv(join(freeze_data, "Discoveries.txt"))

leni = np.array([len(val["AREABLOCK"].split(",")) for key,val in disco.iterrows()])
indx1 = leni < 2
indxx = leni > 1

disco_1 = disco[indx1].copy()
disco_m = disco[indxx].copy()

unpacked = []
for key, val in disco_m.iterrows():    
    for area_block in val["AREABLOCK"].split(","):
        _new = val.copy()
        _new["AREABLOCK"] = area_block
        unpacked.append(_new)
    
result = pd.DataFrame(unpacked)
  
disco = pd.concat([disco_1, result])
disco.rename(columns={"AREABLOCK" : "AREABLK", 
                      "LEASENUM" : "Lease Number"}, inplace=True)

"""
Something strange here:
"""
macondo = disco[disco["DISC_NICK"] == "MACONDO"].copy()
disco = disco[disco["DISC_NICK"] != "MACONDO"].copy()

macondo.set_index(["AREABLK"], inplace=True)
disco.set_index(["AREABLK"], inplace=True)

In [134]:
len(np.unique(disco.index)), len(disco)

(166, 166)

In [127]:
disco[disco["AREABLK"] == "MC252"]

,FID,Lease Number,DISC_NAME,DISC_NICK,AREABLK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
111,111,G32306,MC252,MACONDO,MC252,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,10/07/2009,03/08/2010,ST,11,[It] was estimated the well started leaking ro...,,20120429,04/29/2012,-88.387374,28.735226
128,128,G35491,MC208,NEIDERMEYER,MC252,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477


In [128]:
disco[disco["DISC_NICK"] == "NEIDERMEYER"]

,FID,Lease Number,DISC_NAME,DISC_NICK,AREABLK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
128,128,G35491,MC208,NEIDERMEYER,MC208,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477
128,128,G35491,MC208,NEIDERMEYER,MC209,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477
128,128,G35491,MC208,NEIDERMEYER,MC252,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477
128,128,G35491,MC208,NEIDERMEYER,MC253,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477


In [129]:
disco[disco["DISC_NICK"] == "MACONDO"]

,FID,Lease Number,DISC_NAME,DISC_NICK,AREABLK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
111,111,G32306,MC252,MACONDO,MC252,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,10/07/2009,03/08/2010,ST,11,[It] was estimated the well started leaking ro...,,20120429,04/29/2012,-88.387374,28.735226


In [121]:
np.count(np.unique(disco.index))

AttributeError: module 'numpy' has no attribute 'count'

In [116]:
disco.xs("G34984")

,FID,DISC_NAME,DISC_NICK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,MIDEUR_BOE,MAXEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
AREABLK,,,,,,,,,,,,,,,,,,,,,
GC519,139,GC519,WARRIOR/TIMON,http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,0,0,...,,,,31,The Warrior discovery is located approximately...,,20170728,07/28/2017,-90.219304,27.428828
GC563,139,GC519,WARRIOR/TIMON,http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,0,0,...,,,,31,The Warrior discovery is located approximately...,,20170728,07/28/2017,-90.219304,27.428828


In [112]:
help(result.xs)

Help on method xs in module pandas.core.generic:

xs(key, axis=0, level=None, drop_level=True) method of pandas.core.frame.DataFrame instance
    Return cross-section from the Series/DataFrame.
    
    This method takes a `key` argument to select data at a particular
    level of a MultiIndex.
    
    Parameters
    ----------
    key : label or tuple of label
        Label contained in the index, or partially in a MultiIndex.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis to retrieve cross-section on.
    level : object, defaults to first n levels (n=1 or len(key))
        In case of a key partially contained in a MultiIndex, indicate
        which levels are used. Levels can be referred by label or position.
    drop_level : bool, default True
        If False, returns object with same levels as self.
    
    Returns
    -------
    Series or DataFrame
        Cross-section from the original Series or DataFrame
        corresponding to the selected index levels.

In [79]:
disco_m

,FID,LEASENUM,DISC_NAME,DISC_NICK,AREABLOCK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
0,0,G16783,GC727,CAESAR TONGA,"GC726, GC727, GC770, GC683",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,GC640,200,...,07/02/2003,10/28/2003,PA,7,,,20050822,08/22/2005,-90.833332,27.243621
33,33,G12166,MC765,PRINCESS,"MC765, MC766",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC807,0,...,07/22/2002,09/04/2002,COM,5,Princess is Shell's first Gulf of Mexico devel...,Princess includes an intervention system locat...,20020213,02/13/2002,-89.093596,28.203328
56,56,G21738,EW871,EW871,"EW871, EW872",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EW914,0,...,02/12/2001,03/10/2001,COM,8,EW914 is classified as depleted,,20020812,08/12/2002,-90.275729,28.105105
57,57,G21645,EI343,EI343,"EI343, EI344",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EI342,0,...,06/08/2003,,DRL,0,Engineering has begun on the design of a tripo...,,20030811,08/11/2003,-91.426559,28.192847
94,94,G27278,MC519,SANTA CRUZ/SANTIAGO,"MC519, MC563",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC607,0,...,,,,13,Open-hole logging indicated >140ft of net gas ...,Noble operates Santa Cruz with 23.25% working ...,20090402,04/02/2009,-88.236020,28.475236
115,115,G31943,WR095,YUCATAN NORTH,"WR095 , WR096, WR139 , WR140",http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,...,07/21/2012,,,21,INPEX holds a 15% participating interest and ...,,20130806,08/06/2013,-91.593640,26.861107
125,125,G27982,EW834,COELACANTH/HUMMINGBIRD,"EW834,EW790,EW835,MC793",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EW878,64,...,03/26/2010,,TA,15,The field will produce from two geo-pressured ...,Another development well will capture a proven...,20151202,12/02/2015,-89.937492,28.156996
126,126,G24060,MC214,ODD JOB,"MC214,MC215",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC214,0,...,11/08/2014,12/16/2014,ST,8,Technip will use its Deep Blue ultra-deepwater...,,20151218,12/18/2015,-88.066508,28.781713
128,128,G35491,MC208,NEIDERMEYER,"MC208,MC209,MC252,MC253",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477
139,139,G34984,GC519,WARRIOR/TIMON,"GC519, GC563",http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,...,,,,31,The Warrior discovery is located approximately...,,20170728,07/28/2017,-90.219304,27.428828


In [80]:
disco.xs("G16783")

KeyError: 'G16783'

In [66]:
disco

,FID,LEASENUM,DISC_NAME,DISC_NICK,AREABLOCK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
1,1,G24922,SS097,BETHPAGE,SS097,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,SS097,0,...,,,,3,The SS097 #1 well was designed to test both a ...,This is an extension to a depleted field.,20050815,08/15/2005,-91.016541,28.882762
2,2,G24924,SS111,LAKER,SS111,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,SS111,0,...,07/12/1993,08/31/1993,PA,8,The #2 well was drilled to 13500 ft to test a ...,The Company plans to drill two additional well...,20050815,08/15/2005,-90.967943,28.844243
3,3,G22246,HI119,BALD EAGLE NORTH,HI119,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Deep,HI129,0,...,11/09/2004,12/10/2004,COM,14,,,20050815,08/15/2005,-93.796289,29.300233
4,4,G24730,WC295,FASTBALL,WC295,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,,,0,...,04/01/2005,05/30/2005,DRL,6,A production liner currently is being run for ...,Each pay zone was encountered at the predicted...,20050725,07/25/2005,-93.682500,29.302318
5,5,G26315,GC512,STAMPEDE/KNOTTY HEAD,GC512,http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,GC468,0,...,03/14/2005,03/23/2005,ST,4,"Wire-line logging, fluid sampling, pressure me...",Drilling continues towards the primary objecti...,20050711,07/11/2005,-90.560416,27.456999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,115,G31943,WR095,YUCATAN NORTH,"WR095 , WR096, WR139 , WR140",http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,...,07/21/2012,,,21,INPEX holds a 15% participating interest and ...,,20130806,08/06/2013,-91.593640,26.861107
125,125,G27982,EW834,COELACANTH/HUMMINGBIRD,"EW834,EW790,EW835,MC793",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EW878,64,...,03/26/2010,,TA,15,The field will produce from two geo-pressured ...,Another development well will capture a proven...,20151202,12/02/2015,-89.937492,28.156996
126,126,G24060,MC214,ODD JOB,"MC214,MC215",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC214,0,...,11/08/2014,12/16/2014,ST,8,Technip will use its Deep Blue ultra-deepwater...,,20151218,12/18/2015,-88.066508,28.781713
128,128,G35491,MC208,NEIDERMEYER,"MC208,MC209,MC252,MC253",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477


In [60]:
disco_m

,FID,LEASENUM,DISC_NAME,DISC_NICK,AREABLOCK,SOURCE,DISC_TYPE,EXT_TYPE,EXTFIELD,MINEUR_BOE,...,DISSPUDDT,DISTDDATE,DISSTATUS,PLANWELLS,COMMENT_1,COMMENT_2,ESADATE,ESADATE2,X_COORD,Y_COORD
0,0,G16783,GC727,CAESAR TONGA,"GC726, GC727, GC770, GC683",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,GC640,200,...,07/02/2003,10/28/2003,PA,7,,,20050822,08/22/2005,-90.833332,27.243621
33,33,G12166,MC765,PRINCESS,"MC765, MC766",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC807,0,...,07/22/2002,09/04/2002,COM,5,Princess is Shell's first Gulf of Mexico devel...,Princess includes an intervention system locat...,20020213,02/13/2002,-89.093596,28.203328
56,56,G21738,EW871,EW871,"EW871, EW872",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EW914,0,...,02/12/2001,03/10/2001,COM,8,EW914 is classified as depleted,,20020812,08/12/2002,-90.275729,28.105105
57,57,G21645,EI343,EI343,"EI343, EI344",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EI342,0,...,06/08/2003,,DRL,0,Engineering has begun on the design of a tripo...,,20030811,08/11/2003,-91.426559,28.192847
94,94,G27278,MC519,SANTA CRUZ/SANTIAGO,"MC519, MC563",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC607,0,...,,,,13,Open-hole logging indicated >140ft of net gas ...,Noble operates Santa Cruz with 23.25% working ...,20090402,04/02/2009,-88.236020,28.475236
115,115,G31943,WR095,YUCATAN NORTH,"WR095 , WR096, WR139 , WR140",http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,...,07/21/2012,,,21,INPEX holds a 15% participating interest and ...,,20130806,08/06/2013,-91.593640,26.861107
125,125,G27982,EW834,COELACANTH/HUMMINGBIRD,"EW834,EW790,EW835,MC793",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,EW878,64,...,03/26/2010,,TA,15,The field will produce from two geo-pressured ...,Another development well will capture a proven...,20151202,12/02/2015,-89.937492,28.156996
126,126,G24060,MC214,ODD JOB,"MC214,MC215",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC214,0,...,11/08/2014,12/16/2014,ST,8,Technip will use its Deep Blue ultra-deepwater...,,20151218,12/18/2015,-88.066508,28.781713
128,128,G35491,MC208,NEIDERMEYER,"MC208,MC209,MC252,MC253",http://gom.earthsci.com/Website/DiscProsp/Disc...,Extension Discovery,New Pool Other,MC296,0,...,11/30/2012,02/03/2013,COM,0,"Well #1 & #3 Producing April 2015, Well #2 Pro...",,20160615,06/15/2016,-88.362863,28.757477
139,139,G34984,GC519,WARRIOR/TIMON,"GC519, GC563",http://gom.earthsci.com/Website/DiscProsp/Disc...,New Field Discovery,,,0,...,,,,31,The Warrior discovery is located approximately...,,20170728,07/28/2017,-90.219304,27.428828


In [25]:
disco["AREABLOCK"].split(",")

AttributeError: 'Series' object has no attribute 'split'

In [24]:
disco.loc[1,"AREABLOCK"].split(",")

['SS097']

In [2]:
well_headers = pd.read_csv(join(freeze_data, "Well_Header_GOM3.txt"), low_memory=False)
indx = well_headers["FABBRV"].str.match("[A-Z]{1,}[0-9]{3,3}")
well_headers[indx].copy()

,FID,APIHEADER,WELLNAME,FABBRV,SPUDDATE,WBHLSENUM,WRKBELEV,WBHMD,WBHTVD,WSRFNSDIST,...,WOPRODLY,WGPRODLY,WGEPRODLY,WWPRODLY,WEPRODLY,WCUMGOR,NUMLOGS,WSFLSENUM,PCOMPLEXID,PSTRUCNUM
4,4,1707502442,002,WD027,19600715,S00192,46,15653,15653,10835,...,0,0,0,0,0,154.697178,0,S00192,20256,1
5,5,1707502444,A002,WD027,19530401,S00192,63,14150,13723,3753,...,0,0,0,0,0,0.000000,0,S00192,,
6,6,1707502448,009,WD027,19630708,S00192,57,16336,16336,8840,...,0,0,0,0,0,0.000000,0,S00192,,
7,7,1707502450,003,WD027,19631219,S00192,62,16450,16450,10800,...,0,0,0,0,0,0.000000,0,S00192,,
8,8,1707502451,001,WD027,19600501,S00192,45,15500,15500,5675,...,0,0,0,0,0,0.000000,0,S00192,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37880,37880,6082440003,001,LL399,20040522,G23480,75,27775,27721,734,...,0,0,0,0,0,0.000000,49,G23480,,
37881,37881,6082440004,SS001,LL001,20041209,G10486,79,17492,16101,5444,...,0,0,0,0,0,4926.492698,45,G10487,1766,1
37883,37883,6082440006,SS002,LL399,20050714,G23480,82,18613,18582,5429,...,0,0,0,0,0,3492.079629,17,G23480,1766,1
37885,37885,6082440008,SS003,LL399,20080111,G23480,80,19344,17891,5264,...,0,0,0,0,0,3649.124781,12,G23480,1766,1


0        False
1        False
2        False
3        False
4         True
         ...  
37902    False
37903    False
37904    False
37905    False
37906    False
Name: FABBRV, Length: 37907, dtype: bool

In [2]:
block_data = get_blocks(base_directory, True)
prod_by_lease = load_production_data(base_directory)
regional_prod = load_regional_prod(base_directory)

In [4]:
prod_by_lease

,Production Month,Production Year,Lease Oil Production (BBL),Lease Condensate Production (BBL),Lease Gas-Well-Gas Production (MCF),Lease Oil-Well-Gas Production (MCF),Lease Water Production (BBL),Producing Completions,Lease Max Water Depth (meters),Production Date
Lease Number,,,,,,,,,,
G23243,10,2009,0,0,0,0,0,0,1174,2009-10-01
G23243,1,2009,0,0,0,0,0,0,1174,2009-01-01
G23243,5,2009,0,0,0,0,0,0,1174,2009-05-01
G23243,11,2009,0,0,0,0,0,0,1174,2009-11-01
G23243,12,2009,0,0,0,0,0,0,1174,2009-12-01
...,...,...,...,...,...,...,...,...,...,...
G35823,9,2019,0,0,0,0,0,0,1972,2019-09-01
G35823,10,2019,0,0,0,0,0,0,1972,2019-10-01
G35823,11,2019,0,0,0,0,0,0,1972,2019-11-01


In [26]:
def production_dates(prod):
    month = prod["Production Month"].values.astype(int)
    year = prod["Production Year"].values.astype(int)
    return [pd.datetime(i, j, 1) for i,j in zip(year, month)]

boem_production_by_block_dtypes = {
        "Production Month" : int,
        "Production Year" : int,
        "Lease Oil Production (BBL)" : int,
        "Lease Condensate Production (BBL)" : int,
        "Lease Gas-Well-Gas Production (MCF)" : int,
        "Lease Oil-Well-Gas Production (MCF)" : int,   
        "Lease Water Production (BBL)" : int, 
        "Producing Completions" : int, 
        "Lease Max Water Depth (meters)" : int}

boem_production_by_block = pd.read_csv(join(base_directory, "ProdData2007-2010.csv"))
indx = np.logical_or(boem_production_by_block["Production Month"].isna(),
                     boem_production_by_block["Production Month"].isna())
boem_production_by_block.drop(boem_production_by_block.index[indx], inplace=True)
boem_production_by_block["Production Date"] = production_dates(boem_production_by_block)

for key, val in boem_production_by_block_dtypes.items():
    boem_production_by_block[key] = boem_production_by_block[key].astype(val)


In [27]:
boem_production_by_block.dtypes

Lease Number                                   object
Production Month                                int64
Production Year                                 int64
Lease Oil Production (BBL)                      int64
Lease Condensate Production (BBL)               int64
Lease Gas-Well-Gas Production (MCF)             int64
Lease Oil-Well-Gas Production (MCF)             int64
Lease Water Production (BBL)                    int64
Producing Completions                           int64
Lease Max Water Depth (meters)                  int64
Production Date                        datetime64[ns]
dtype: object

In [72]:
prod = boem_production_by_block
mn = prod["Production Month"].values.astype(int)
yr = prod["Production Year"].values.astype(int)

In [74]:
yr.shape

(66265,)

In [75]:
[pd.datetime(i, j, 1) for i,j in zip(yr,mn)]

[datetime.datetime(2009, 10, 1, 0, 0),
 datetime.datetime(2009, 1, 1, 0, 0),
 datetime.datetime(2009, 5, 1, 0, 0),
 datetime.datetime(2009, 11, 1, 0, 0),
 datetime.datetime(2009, 12, 1, 0, 0),
 datetime.datetime(2009, 2, 1, 0, 0),
 datetime.datetime(2009, 3, 1, 0, 0),
 datetime.datetime(2009, 4, 1, 0, 0),
 datetime.datetime(2009, 9, 1, 0, 0),
 datetime.datetime(2009, 7, 1, 0, 0),
 datetime.datetime(2009, 8, 1, 0, 0),
 datetime.datetime(2009, 6, 1, 0, 0),
 datetime.datetime(2009, 11, 1, 0, 0),
 datetime.datetime(2009, 3, 1, 0, 0),
 datetime.datetime(2009, 2, 1, 0, 0),
 datetime.datetime(2009, 10, 1, 0, 0),
 datetime.datetime(2009, 9, 1, 0, 0),
 datetime.datetime(2009, 6, 1, 0, 0),
 datetime.datetime(2009, 7, 1, 0, 0),
 datetime.datetime(2009, 8, 1, 0, 0),
 datetime.datetime(2009, 5, 1, 0, 0),
 datetime.datetime(2009, 1, 1, 0, 0),
 datetime.datetime(2009, 4, 1, 0, 0),
 datetime.datetime(2009, 12, 1, 0, 0),
 datetime.datetime(2009, 2, 1, 0, 0),
 datetime.datetime(2009, 4, 1, 0, 0),
 datet

In [35]:
def production_dates(prod):
    prod[prod == "Month Total"] = ""
    prod = pd.to_datetime(prod)
    return prod

boem_production_dates = ["Production Month/Year"]
production_by_area = pd.read_csv(join(base_directory, "PBPA.csv"), 
                                 parse_dates=boem_production_dates,
                                 date_parser=production_dates)
                                 

In [36]:
production_by_area

,Production Month/Year,Planning Area,Oil,Condensate,Total Oil (BBLS),MMBBL/Day,Gas,Casinghead,Total Gas (MCF),BCF/DAY
0,1947-11-01,Central GOM,7082,0,7082,0.000,0,795,795,0.000
1,NaT,NaN,7082,0,7082,0.000,0,795,795,0.000
2,1947-12-01,Central GOM,7082,0,7082,0.000,0,795,795,0.000
3,NaT,NaN,7082,0,7082,0.000,0,795,795,0.000
4,1948-01-01,Central GOM,7082,0,7082,0.000,0,795,795,0.000
...,...,...,...,...,...,...,...,...,...,...
2428,2019-11-01,Western GOM,2488639,128589,2617228,0.087,2358938,3458457,5817395,0.194
2429,NaT,NaN,52745127,2824722,55569849,1.852,26992430,54197744,81190174,2.706
2430,2019-12-01,Central GOM,2157583,126955,2284538,0.074,2613229,2873200,5486429,0.177
2431,2019-12-01,Western GOM,67068,35794,102862,0.003,1171592,83802,1255394,0.040


In [26]:
production_dates(production_by_area)

,Production Month/Year,Planning Area,Oil,Condensate,Total Oil (BBLS),MMBBL/Day,Gas,Casinghead,Total Gas (MCF),BCF/DAY
0,1947-11-01,Central GOM,7082,0,7082,0,0,795,795,0
1,NaT,,,,,,,,,
2,1947-12-01,Central GOM,7082,0,7082,0,0,795,795,0
3,NaT,,,,,,,,,
4,1948-01-01,Central GOM,7082,0,7082,0,0,795,795,0
...,...,...,...,...,...,...,...,...,...,...
2428,2019-11-01,Western GOM,2488639,128589,2617228,0.087,2358938,3458457,5817395,0.194
2429,NaT,,,,,,,,,
2430,2019-12-01,Central GOM,2157583,126955,2284538,0.074,2613229,2873200,5486429,0.177
2431,2019-12-01,Western GOM,67068,35794,102862,0.003,1171592,83802,1255394,0.04


0      1947-11-01
1             NaT
2      1947-12-01
3             NaT
4      1948-01-01
          ...    
2428   2019-11-01
2429          NaT
2430   2019-12-01
2431   2019-12-01
2432          NaT
Name: Production Month/Year, Length: 2433, dtype: datetime64[ns]

In [12]:
lease_data[lease_data["LEASENUM"]=="G36684"]

,LEASENUM,SERTYPECD,SALENUM,EXPCTEXPDT,APISTCTYCD,TRACTNUM,LSEEFFCTDT,PRIMRYTERM,EXPIREDT,BIDSYSTMCD,...,Own5,Conum5,Asgnpct5,LASTPASTF,NUMOWNER,NUMOP,NUMALIQUOT,HASAOR,DESC_,geometry
0,G36684,Oil/Gas/Salt/Sulphur,252,20290531,60824,36684,20190601,10,0,R23,...,None,None,0.0,None,1,1,0,None,All of Block,"POLYGON ((-87.71344 27.75986, -87.76243 27.759..."
